In [ ]:
# 컨텐트 기반 필터링(Content based filtering) : 사용자가 특정 아이템을 선호하는 경우 그 아이템과 비슷한 컨텐츠를 가진 다른 아이템을 추천하는 방식

import pandas as pd

df = pd.read_csv('c:/data/movies/tmdb_5000_movies.csv')

df.head()

In [ ]:
# genres : 영화 장르
# keywords : 영화의 키워드
# original_language : 영화 언어
# title : 제목
# vote_average : 평점 평균
# vote_count : 평점 카운트
# popularity : 인기도
# overview : 줄거리
# 필요한 필드만 선택

df = df[['id','genres', 'vote_average', 'vote_count','popularity','title',  
             'keywords', 'overview']]

df

In [ ]:
df[['genres', 'keywords']].head()

In [ ]:
dict1 = "{'a': 3, 'b': 5}"

print(type(dict1)) #스트링

#print(dict1['a']) #스트링이므로 에러가 발생함
#print(dict1['b'])

In [ ]:
dict2 = eval(dict1) #자료형을 스트링에서 딕셔너리로 변환

print(type(dict2))  
print(dict2['a'])
print(dict2['b'])

In [ ]:
# eval('문자열') - 문자열 내에 포함된 파이썬 명령어를 실행하는 함수
df['genres'] = df['genres'].apply(eval)
df['keywords'] = df['keywords'].apply(eval)

# 딕셔너리 내부의 하위 변수들을 합쳐서 문자열 변수 1개에 저장

df['genres'] = df['genres'].apply(
    lambda x : [d['name'] for d in x]).apply(lambda x : " ".join(x))

df['keywords'] = df['keywords'].apply(
    lambda x : [d['name'] for d in x]).apply(lambda x : " ".join(x))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vector = CountVectorizer()

# 장르에 대한 출현빈도 계산

c_vector_genres = count_vector.fit_transform(df['genres']).toarray()

c_vector_genres

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# 코사인 유사도 계산

genre_c_sim = cosine_similarity(c_vector_genres, c_vector_genres).argsort()[:, ::-1]

cosine_similarity(c_vector_genres, c_vector_genres)[0][:30]

In [ ]:
# 영화 추천 함수
def get_recommend_movie_list(data, movie_title, top=5):

    # 특정 영화 검색
    target_movie_index = data[data['title'] == movie_title].index.values

    # 코사인 유사도 상위 5행
    sim_index = genre_c_sim[target_movie_index, :top].reshape(-1)

    # 아이디가 같은 self row 제외
    sim_index = sim_index[sim_index != target_movie_index]

    # data frame으로 만들고 vote_count으로 정렬한 뒤 return
    result = data.iloc[sim_index].sort_values('vote_average', ascending=False)[:10]
    return result

In [ ]:
df[df['title'] == 'Superman']

In [ ]:
get_recommend_movie_list(df, movie_title='Superman')